In [1]:
!pip install torch
!pip install pandas
!pip install datasets
!pip install transformers
!pip install scikit-learn
!pip install peft
!pip install codecarbon
!pip install matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 37.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 25.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 71.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [2]:
import torch
import pandas as pd
from datasets import load_dataset
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from datasets import DatasetDict
from peft import get_peft_model, LoraConfig, TaskType
from transformers import TrainerCallback
import time
from torch.optim import AdamW
from transformers import get_scheduler
import matplotlib.pyplot as plt
from codecarbon import EmissionsTracker

In [3]:
# Inspecte les étiquettes du dataset
from datasets import load_dataset

# Charger le dataset
dataset = load_dataset("hate_speech18")
print(dataset)

# Vérifie les étiquettes dans le dataset d'entraînement
print(dataset['train'].features['label'])


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/5.61k [00:00<?, ?B/s]

hate_speech18.py:   0%|          | 0.00/3.41k [00:00<?, ?B/s]

The repository for hate_speech18 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/hate_speech18.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


data.zip:   0%|          | 0.00/3.19M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/10944 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'user_id', 'subforum_id', 'num_contexts', 'label'],
        num_rows: 10944
    })
})
ClassLabel(names=['noHate', 'hate', 'idk/skip', 'relation'], id=None)


In [4]:
# Séparer en train et test
split_datasets = dataset["train"].train_test_split(test_size=0.2)

datasets = DatasetDict({
    "train": split_datasets["train"],
    "test": split_datasets["test"],
})

# Préparer le tokenizer
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

# Appliquer la tokenization
tokenized_datasets = datasets.map(tokenize_function, batched=True)

# Charger le modèle pré-entraîné sans fine-tuning
base_model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=4)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

Map:   0%|          | 0/8755 [00:00<?, ? examples/s]

Map:   0%|          | 0/2189 [00:00<?, ? examples/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
# Récupérer les noms des classes directement à partir du dataset
class_names = dataset['train'].features['label'].names

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = torch.argmax(torch.tensor(logits), dim=-1)

    accuracy = accuracy_score(labels, predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average=None, zero_division=1)

    # Calcul des métriques globales (moyenne des classes)
    precision_global = precision.mean()
    recall_global = recall.mean()
    f1_macro = f1.mean()  # Ajout du F1-score macro

    metrics = {
        "accuracy": accuracy,
        "precision_global": precision_global,
        "recall_global": recall_global,
        "f1_macro": f1_macro
    }

    # Ajouter les métriques par classe avec les noms explicites
    for i, (p, r, f) in enumerate(zip(precision, recall, f1)):
        metrics[f"precision_{class_names[i]}"] = p
        metrics[f"recall_{class_names[i]}"] = r
        metrics[f"f1_{class_names[i]}"] = f

    return metrics

# Configurer LoRA
lora_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,  # Classification de séquence
    r=8,  # Rang de la décomposition
    lora_alpha=16,  # Facteur d'adaptation
    lora_dropout=0.1,  # Dropout pour LoRA
    target_modules=["q_lin", "v_lin"]  # Modules spécifiques à LoRA dans les transformers
)

# Appliquer LoRA au modèle
model = get_peft_model(base_model, lora_config)
model.print_trainable_parameters()

# Configurer l'entraînement avec fine-tuning LoRA
training_args_lora = TrainingArguments(
    output_dir="./results_lora",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="steps",
    logging_steps=100,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=2,
    num_train_epochs=5,
    learning_rate=5e-5,
    weight_decay=0.05,
    lr_scheduler_type="linear",
    warmup_steps=2000,
    max_grad_norm=1,
    metric_for_best_model="f1_macro",
    seed=123,
    fp16=True,
    report_to="none",
    overwrite_output_dir=True,
    disable_tqdm=False,
    logging_first_step=True,
)

# Initialisation du Trainer
trainer_lora = Trainer(
    model=model,
    args=training_args_lora,
    train_dataset=tokenized_datasets["train"],  # Dataset d'entraînement
    eval_dataset=tokenized_datasets["test"],  # Dataset de test
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

tracker = EmissionsTracker()
tracker.start()

train_result_lora = trainer_lora.train()

emissions = tracker.stop()


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-5-731d5aa5eb00>:72: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer_lora = Trainer(


trainable params: 741,124 || all params: 67,697,672 || trainable%: 1.0948


No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
[codecarbon INFO @ 12:40:26] [setup] RAM Tracking...
[codecarbon INFO @ 12:40:26] [setup] CPU Tracking...
[codecarbon WARNING @ 12:40:26] No CPU tracking mode found. Falling back on CPU constant mode. 
 Linux OS detected: Please ensure RAPL files exist at \sys\class\powercap\intel-rapl to measure CPU

[codecarbon WARNING @ 12:40:27] We saw that you have a Intel(R) Xeon(R) CPU @ 2.00GHz but we don't know it. Please contact us.
[codecarbon INFO @ 12:40:27] CPU Model on constant consumption mode: Intel(R) Xeon(R) CPU @ 2.00GHz
[codecarbon INFO @ 12:40:27] [setup] GPU Tracking...
[codecarbon INFO @ 12:40:27] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 12:40:27] >>> Tracker's metadata:
[codecarbon INFO @ 12:40:27]   P

Epoch,Training Loss,Validation Loss,Accuracy,Precision Global,Recall Global,F1 Macro,Precision Nohate,Recall Nohate,F1 Nohate,Precision Hate,Recall Hate,F1 Hate,Precision Idk/skip,Recall Idk/skip,F1 Idk/skip,Precision Relation,Recall Relation,F1 Relation
1,0.458100,0.442724,0.867519,0.966880,0.250000,0.232265,0.867519,1.000000,0.929061,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000
2,0.366800,0.366937,0.879397,0.930540,0.282405,0.291051,0.880056,0.996840,0.934815,0.842105,0.132780,0.229391,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000
3,0.372100,0.346769,0.882595,0.903506,0.302346,0.318392,0.887996,0.989468,0.935990,0.726027,0.219917,0.337580,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000
4,0.299400,0.314577,0.888534,0.885710,0.425809,0.482003,0.907080,0.971564,0.938215,0.635762,0.398340,0.489796,1.000000,0.333333,0.500000,1.000000,0.000000,0.000000


[codecarbon INFO @ 12:40:42] Energy consumed for RAM : 0.000020 kWh. RAM Power : 4.7530388832092285 W
[codecarbon INFO @ 12:40:42] Energy consumed for all CPUs : 0.000177 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 12:40:42] Energy consumed for all GPUs : 0.000250 kWh. Total GPU Power : 59.83634941638935 W
[codecarbon INFO @ 12:40:42] 0.000447 kWh of electricity used since the beginning.
[codecarbon INFO @ 12:40:57] Energy consumed for RAM : 0.000040 kWh. RAM Power : 4.7530388832092285 W
[codecarbon INFO @ 12:40:57] Energy consumed for all CPUs : 0.000354 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 12:40:57] Energy consumed for all GPUs : 0.000521 kWh. Total GPU Power : 65.31675293844282 W
[codecarbon INFO @ 12:40:57] 0.000915 kWh of electricity used since the beginning.
[codecarbon INFO @ 12:41:12] Energy consumed for RAM : 0.000059 kWh. RAM Power : 4.7530388832092285 W
[codecarbon INFO @ 12:41:12] Energy consumed for all CPUs : 0.000531 kWh. Total CPU Power : 42.5 W
[codeca

In [6]:
import torch
import pandas as pd
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from codecarbon import EmissionsTracker

# Charger le modèle de base
base_model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=4)

# ✅ CHOIX : Sélectionner les paramètres à entraîner
freeze_mode = "last_layers"  # "head", "last_layers", "embeddings", "none"

if freeze_mode == "head":
    for param in base_model.distilbert.parameters():
        param.requires_grad = False  # Gèle tout sauf la tête

elif freeze_mode == "last_layers":
    for param in base_model.distilbert.parameters():
        param.requires_grad = False  # Gèle tout
    for layer in base_model.distilbert.transformer.layer[-2:]:  # Dégeler les 2 dernières couches
        for param in layer.parameters():
            param.requires_grad = True

elif freeze_mode == "embeddings":
    for param in base_model.distilbert.embeddings.parameters():
        param.requires_grad = False  # Gèle uniquement les embeddings

elif freeze_mode == "none":
    pass  # Fine-tune complet

# Vérifier les paramètres entraînables
for name, param in base_model.named_parameters():
    print(f"{name}: requires_grad = {param.requires_grad}")

# Fonction de calcul des métriques
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = torch.argmax(torch.tensor(logits), dim=-1)

    accuracy = accuracy_score(labels, predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average=None, zero_division=1)

    return {
        "accuracy": accuracy,
        "precision_global": precision.mean(),
        "recall_global": recall.mean(),
        "f1_macro": f1.mean()
    }

# Arguments d'entraînement
training_args = TrainingArguments(
    output_dir="./results_sans_lora",
    evaluation_strategy="epoch",
    save_strategy="no",
    logging_strategy="steps",
    logging_steps=100,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=2,
    num_train_epochs=5,
    learning_rate=5e-5,
    weight_decay=0.05,
    lr_scheduler_type="linear",
    warmup_steps=2000,
    max_grad_norm=1,
    metric_for_best_model="f1_macro",
    seed=123,
    fp16=True,
    report_to="none",
    overwrite_output_dir=True,
    disable_tqdm=False,
    logging_first_step=True,
)


# Initialisation du Trainer
trainer = Trainer(
    model=base_model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],  # Dataset d'entraînement
    eval_dataset=tokenized_datasets["test"],  # Dataset de test
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# Suivi de l'empreinte carbone
tracker = EmissionsTracker()
tracker.start()

# Entraînement
train_result = trainer.train()

# Fin du suivi carbone
emissions = tracker.stop()


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-6-02bd8b05d931>:77: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


distilbert.embeddings.word_embeddings.weight: requires_grad = False
distilbert.embeddings.position_embeddings.weight: requires_grad = False
distilbert.embeddings.LayerNorm.weight: requires_grad = False
distilbert.embeddings.LayerNorm.bias: requires_grad = False
distilbert.transformer.layer.0.attention.q_lin.weight: requires_grad = False
distilbert.transformer.layer.0.attention.q_lin.bias: requires_grad = False
distilbert.transformer.layer.0.attention.k_lin.weight: requires_grad = False
distilbert.transformer.layer.0.attention.k_lin.bias: requires_grad = False
distilbert.transformer.layer.0.attention.v_lin.weight: requires_grad = False
distilbert.transformer.layer.0.attention.v_lin.bias: requires_grad = False
distilbert.transformer.layer.0.attention.out_lin.weight: requires_grad = False
distilbert.transformer.layer.0.attention.out_lin.bias: requires_grad = False
distilbert.transformer.layer.0.sa_layer_norm.weight: requires_grad = False
distilbert.transformer.layer.0.sa_layer_norm.bias: 

[codecarbon INFO @ 14:03:30] [setup] RAM Tracking...
[codecarbon INFO @ 14:03:30] [setup] CPU Tracking...
[codecarbon WARNING @ 14:03:30] No CPU tracking mode found. Falling back on CPU constant mode. 
 Linux OS detected: Please ensure RAPL files exist at \sys\class\powercap\intel-rapl to measure CPU

[codecarbon WARNING @ 14:03:32] We saw that you have a Intel(R) Xeon(R) CPU @ 2.00GHz but we don't know it. Please contact us.
[codecarbon INFO @ 14:03:32] CPU Model on constant consumption mode: Intel(R) Xeon(R) CPU @ 2.00GHz
[codecarbon INFO @ 14:03:32] [setup] GPU Tracking...
[codecarbon INFO @ 14:03:32] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 14:03:32] >>> Tracker's metadata:
[codecarbon INFO @ 14:03:32]   Platform system: Linux-6.1.85+-x86_64-with-glibc2.35
[codecarbon INFO @ 14:03:32]   Python version: 3.11.11
[codecarbon INFO @ 14:03:32]   CodeCarbon version: 2.8.3
[codecarbon INFO @ 14:03:32]   Available RAM : 12.675 GB
[codecarbon INFO @ 14:03:32]   CPU count: 2
[codeca

Epoch,Training Loss,Validation Loss,Accuracy,Precision Global,Recall Global,F1 Macro
1,0.373300,0.337505,0.882595,0.885689,0.371970,0.419771
2,0.313200,0.328007,0.889447,0.800433,0.510542,0.528347
3,0.254400,0.327872,0.884879,0.841347,0.550997,0.565888
4,0.189300,0.392707,0.894929,0.858231,0.536467,0.566809


[codecarbon INFO @ 14:03:47] Energy consumed for RAM : 0.000020 kWh. RAM Power : 4.7530388832092285 W
[codecarbon INFO @ 14:03:47] Energy consumed for all CPUs : 0.000177 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 14:03:47] Energy consumed for all GPUs : 0.000246 kWh. Total GPU Power : 58.84944371245871 W
[codecarbon INFO @ 14:03:47] 0.000443 kWh of electricity used since the beginning.
[codecarbon INFO @ 14:04:02] Energy consumed for RAM : 0.000040 kWh. RAM Power : 4.7530388832092285 W
[codecarbon INFO @ 14:04:02] Energy consumed for all CPUs : 0.000354 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 14:04:02] Energy consumed for all GPUs : 0.000521 kWh. Total GPU Power : 66.12177712657102 W
[codecarbon INFO @ 14:04:02] 0.000915 kWh of electricity used since the beginning.
[codecarbon INFO @ 14:04:17] Energy consumed for RAM : 0.000059 kWh. RAM Power : 4.7530388832092285 W
[codecarbon INFO @ 14:04:17] Energy consumed for all CPUs : 0.000531 kWh. Total CPU Power : 42.5 W
[codeca

In [7]:
import pandas as pd

def generate_metrics_report(trainer, final_metrics, training_args, emissions, tokenizer=None, lora_config=None):
    """
    Génère un rapport compact avec les métriques clés.
    Affiche automatiquement les infos liées à LoRA si `lora_config` est fourni.
    """
    def safe_format(value):
        return f"{value:.4f}" if isinstance(value, (int, float)) else str(value)

    history = pd.DataFrame(trainer.state.log_history)

    # Calcul du temps moyen par époque
    if 'epoch' in history.columns and 'train_runtime' in history.columns:
        # Filtrer les lignes liées aux époques
        epoch_logs = history[history['epoch'].notna()]
        if not epoch_logs.empty:
            # Calcul du temps total par époque
            epoch_times = epoch_logs.groupby('epoch')['train_runtime'].sum()
            mean_epoch_time = epoch_times.mean() if not epoch_times.empty else "N/A"
        else:
            mean_epoch_time = "N/A"
    else:
        mean_epoch_time = "N/A"

    # Section LoRA (optionnelle)
    lora_section = (
        f"- **LoRA Config** : r={lora_config.r}, alpha={lora_config.lora_alpha}, dropout={lora_config.lora_dropout}"
        if lora_config else "- **LoRA** : non utilisé"
    )

    # Choix du nom de fichier / courbe
    suffix = "_lora" if lora_config else "_sans_lora"
    report_path = f"./results{suffix}/training_report{suffix}.txt"
    learning_curve_path = f"./results{suffix}/learning_curves{suffix}.png"

    report = f"""
# 📊 Rapport d'entraînement et d'évaluation

## 🔍 **Résultats**
- **Accuracy** : {safe_format(final_metrics.get('eval_accuracy', 'N/A'))}
- **F1-score (macro)** : {safe_format(final_metrics.get('eval_f1_macro', 'N/A'))}
- **Precision globale** : {safe_format(final_metrics.get('eval_precision_global', 'N/A'))}
- **Recall global** : {safe_format(final_metrics.get('eval_recall_global', 'N/A'))}
- **Loss finale** : {safe_format(final_metrics.get('eval_loss', 'N/A'))}

## ⚙️ **Hyperparamètres**
- **Epochs** : {training_args.num_train_epochs}
- **Batch Size (train / eval)** : {training_args.per_device_train_batch_size} / {training_args.per_device_eval_batch_size}
- **Gradient Accumulation Steps** : {training_args.gradient_accumulation_steps}
- **Learning Rate** : {training_args.learning_rate}
- **Weight Decay** : {training_args.weight_decay}
- **Warmup Steps** : {training_args.warmup_steps}
- **Scheduler** : {training_args.lr_scheduler_type}
- **Max Grad Norm** : {training_args.max_grad_norm}
- **Seed** : {training_args.seed}
{lora_section}

## 🧠 **Tokenizer**
- **Tokenizer utilisé** : {getattr(tokenizer, 'name_or_path', 'Non spécifié') if tokenizer else 'Non fourni'}

## ⏱ **Temps d'entraînement**
- **Temps moyen par époque** : {safe_format(mean_epoch_time)} sec

## 🌱 **Empreinte carbone**
- **CO₂ estimé** : {safe_format(emissions)} kg

"""
    with open(report_path, "w") as f:
        f.write(report)
    print(report)


In [8]:
final_metrics = trainer.evaluate()
# final_metrics_lora = trainer_lora.evaluate()
# generate_metrics_report(trainer_lora, final_metrics_lora, training_args_lora, emissions, tokenizer, lora_config)
generate_metrics_report(trainer, final_metrics, training_args, emissions, tokenizer)
print(final_metrics)


# 📊 Rapport d'entraînement et d'évaluation

## 🔍 **Résultats**
- **Accuracy** : 0.8949
- **F1-score (macro)** : 0.5668
- **Precision globale** : 0.8582
- **Recall global** : 0.5365
- **Loss finale** : 0.3927

## ⚙️ **Hyperparamètres**
- **Epochs** : 5
- **Batch Size (train / eval)** : 8 / 8
- **Gradient Accumulation Steps** : 2
- **Learning Rate** : 5e-05
- **Weight Decay** : 0.05
- **Warmup Steps** : 2000
- **Scheduler** : SchedulerType.LINEAR
- **Max Grad Norm** : 1
- **Seed** : 123
- **LoRA** : non utilisé

## 🧠 **Tokenizer**
- **Tokenizer utilisé** : distilbert-base-uncased

## ⏱ **Temps d'entraînement**
- **Temps moyen par époque** : 10.6467 sec

## 🌱 **Empreinte carbone**
- **CO₂ estimé** : 0.0053 kg


{'eval_loss': 0.3927065432071686, 'eval_accuracy': 0.8949291914116034, 'eval_precision_global': 0.8582308453491659, 'eval_recall_global': 0.5364666912894708, 'eval_f1_macro': 0.5668088463650784, 'eval_runtime': 8.8303, 'eval_samples_per_second': 247.897, 'eval_steps_per_second': 3

In [9]:
!zip -r mon_dossier.zip ./

  adding: .config/ (stored 0%)
  adding: .config/config_sentinel (stored 0%)
  adding: .config/hidden_gcloud_config_universe_descriptor_data_cache_configs.db (deflated 97%)
  adding: .config/configurations/ (stored 0%)
  adding: .config/configurations/config_default (deflated 15%)
  adding: .config/gce (stored 0%)
  adding: .config/.last_survey_prompt.yaml (stored 0%)
  adding: .config/active_config (stored 0%)
  adding: .config/default_configs.db (deflated 98%)
  adding: .config/.last_update_check.json (deflated 23%)
  adding: .config/logs/ (stored 0%)
  adding: .config/logs/2025.04.07/ (stored 0%)
  adding: .config/logs/2025.04.07/13.42.43.503346.log (deflated 57%)
  adding: .config/logs/2025.04.07/13.42.02.070730.log (deflated 92%)
  adding: .config/logs/2025.04.07/13.42.32.824024.log (deflated 86%)
  adding: .config/logs/2025.04.07/13.42.24.146310.log (deflated 58%)
  adding: .config/logs/2025.04.07/13.42.34.302585.log (deflated 58%)
  adding: .config/logs/2025.04.07/13.42.44.25107

In [11]:
from google.colab import files
files.download("mon_dossier.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>